## Аггрегация разметки датасета RWSD
*Additional testset data edition*

Аггрегация строится по следующей системе:

1. Сбор размеченных пулов с Толоки. Возможны варианты:
    - только общий пул нужно аггрегировать, тогда забирается только он
    - часть данных находится в контрольных заданиях и экзамене, тогда к основному пулу добавляются данные задания
2. Фильтрация разметчиков:
    - в общем пуле есть некоторое количество заранее размеченных заданий - контрольных
    - хорошим считается разметчик, который показывает `accuracy >= 0.5` на данных заданиях
    - формируется список "плохих" разметчиков
3. Аггрегация ответов разметчиков по заданиям:
    - форматирование в заданиях может отличаться от изначального из-за выгрузки с Толоки
    - учитываются только ответы "хороших" разметчиков
    - аггрегация по подготовленным пулам - создается массив карточек вида {key: value}, где key - кортеж из всех значимых элементов задания, value - список из кортежей вида (user_id, answer)
4. Голосование большинством по каждому заданию:
    - минимально необходимое большинство составляет 3 голоса, так как такое большинство валидно для перекрытия 5
    - по результату формируется датафрейм с заданиями и ответами
5. Подгрузка оригинальных данных с разметкой в виде таблицы с заданиями и ответами
6. Соединение таблиц:
    - очистка форматирования в таблице с ответами разметчиков и в таблице с правильными ответами
    - создание единых столбцов с полным заданием
    - соединение таблиц по данному столбцу
    - валидация размеров
7. Подсчет метрик

In [1]:
import pandas as pd
from collections import Counter

### Сбор данных разметки и фильтрация разметчиков

Всего в разметку попали 260 объектов, включая, как "старые" данные (до расширения тестового сета), так и "новые" (добавленные в процессе расширения тестового сета).

Виноград прогонялся двумя пулами. В первый пул попали почти все добавленные объекты. Во второй пул попали 4 новых добавленных объектов (но размер второго пула больше из-за контрольных заданий)

In [2]:
assignments1 = pd.read_csv('assignments_from_pool_41009024__30-08-2023.tsv', sep='\t')
assignments2 = pd.read_csv('assignments_from_pool_41266267__11-09-2023.tsv', sep='\t')

In [3]:
assignments = pd.concat([assignments1, assignments2[assignments1.columns]])

Разметчикам предлагалось на основании имеющихся текст, фраз со словом и с предлогом из данного текста ответить на вопрос, относятся ли обе фразы к одной и той же сущности.

Вход: 
- INPUT:text (пример: `Пршел к Владу и Маше в гости. Она открыла дверь`).
- INPUT:word (пример: `Маша`).
- INPUT:pronoun (пример: `Она`).

Выход:
- OUTPUT:result (булеан: `True` или `False`).

In [4]:
assignments.head(1)

,INPUT:text,INPUT:word,INPUT:pronoun,OUTPUT:result,GOLDEN:result,HINT:text,HINT:default_language,ASSIGNMENT:link,ASSIGNMENT:task_id,ASSIGNMENT:assignment_id,ASSIGNMENT:task_suite_id,ASSIGNMENT:worker_id,ASSIGNMENT:status,ASSIGNMENT:started,ASSIGNMENT:submitted,ASSIGNMENT:accepted
0,Боб заплатил за обучение Чарльза в университет...,Боб,Он,True,True,NaN,NaN,https://platform.toloka.ai/task/41009024/00027...,000271bf80--64ef36132409351a10c5f66d,000271bf80--64ef38197c475157010635d8,000271bf80--64ef38187c475157010635d2,36d93d4f906f48015a04300bb61d8ff1,APPROVED,2023-08-30T12:37:45,2023-08-30T12:38:29.678,2023-08-30T12:38:29.678


Фильтруем толокеров, которые дали меньше половины корректных ответов на контрольных заданиях.

In [5]:
from collections import defaultdict

users_dict = defaultdict(lambda: defaultdict(int))

for idx, row in assignments.iterrows():
    text = row[0]
    word = row[1]
    pron = row[2]

    out = row[3]
    
    gold = row[4]

    user = row[11]

    if str(user) != "nan" and str(gold) != "nan":
        if out == int(gold):
            users_dict[user]["good"] += 1
        else:
            users_dict[user]["bad"] += 1

print("Users total: ", len(users_dict))
bad_users = []
for key, value in users_dict.items():
    percentage_good = value["good"]/(value["good"] + value["bad"])
    if percentage_good < 0.5:
        bad_users.append(key)

print("Bad users:", len(bad_users))

Users total:  155
Bad users: 5


5 из 155 разметчиков на контрольных заданиях показали слишком плохое качество, чтобы учитывать их ответы для расчета метрики.

Теперь нужно оставить только основной пул. Контрольные задания брались из трейна Винограда, потому их нужно убрать. На контрольных заданиях есть `GOLDEN:result`. Также отсеиваем возможные баги Толоки, когда в строке может не быть задания - `INPUT:text` содержит NaN.

In [6]:
assignments_no_control = assignments[assignments['GOLDEN:result'].isnull()]
assignments_no_control_no_null = assignments_no_control[assignments_no_control['INPUT:text'].notnull()]

### Сбор ответов разметчиков и голосование

Собираем ответы голосования большинством для каждого задания.

In [7]:
from collections import defaultdict

text_dict = defaultdict(list)

for text, word, pron, user, out in zip(
    assignments_no_control_no_null["INPUT:text"], assignments_no_control_no_null["INPUT:word"],
    assignments_no_control_no_null["INPUT:pronoun"], assignments_no_control_no_null["ASSIGNMENT:worker_id"], 
    assignments_no_control_no_null["OUTPUT:result"]
    ):
    if user not in bad_users:
        text_dict[(text, word, pron)].append([
                user,
                {"out": out}
        ])

print(len(text_dict))

260


In [8]:
keys = list(text_dict.keys())
Counter([len(text_dict[keys[i]]) for i in range(len(keys))])

Counter({5: 250, 4: 10})

Есть 10 заданий с перектрытием 4. В каждом может быть ситуация 2/2. Такие объекты пропускаем. Если есть согласие (3 голоса большинства), то оставляем, так как для перекрытия 5 порог согласия ровно такой же.

In [9]:
preds_full = {}
for i in range(len(keys)):
    ans = text_dict[keys[i]]
    lst = [ans[j][1]['out'] for j in range(len(ans))]
    cnt = Counter(lst)
    if len(lst) == 5:
        most = Counter([ans[j][1]['out'] for j in range(len(ans))]).most_common(1)[0][1]
        if most >= 3:
            res = Counter([ans[j][1]['out'] for j in range(len(ans))]).most_common(1)[0][0]
            preds_full[keys[i]] = res
    elif len(lst) == 4:
        most = Counter([ans[j][1]['out'] for j in range(len(ans))]).most_common(1)[0][1]
        if most > 2:
            res = Counter([ans[j][1]['out'] for j in range(len(ans))]).most_common(1)[0][0]
            preds_full[keys[i]] = res

In [10]:
len(preds_full)

259

Только в 1 объекте согласованность не была достигнута. Убираем его.

In [12]:
preds_full_df = pd.concat([pd.DataFrame(preds_full.keys(), columns=['text', 'span1', 'span2']), pd.DataFrame(preds_full.values(), columns=['lb'])], axis=1)

### Сопоставление разметки и ground truth

Забираем задания из датасета с правильными метками.

In [13]:
res_df = pd.read_csv('winograd.csv')

После скачивания с Толоки в текстах рушится форматирование, потому нельзя просто сделать join двух табличек. Нужно убрать все "лишнее" форматирование сразу из двух табличек, чтобы остались только тексты, пунктуация и пробелы.

In [14]:
def format_text(text):
    text = (text.strip().replace('\n', ' ').replace('\t', ' ')
            .replace('\r', ' ').replace('  ', ' ').replace('  ', ' ')
            .replace('  ', ' '))
    return text

res_df['text'] = res_df['text'].apply(format_text)
res_df['span1'] = res_df['span1'].apply(format_text)
res_df['span2'] = res_df['span2'].apply(format_text)

preds_full_df['text'] = preds_full_df['text'].apply(format_text)
preds_full_df['span1'] = preds_full_df['span1'].apply(format_text)
preds_full_df['span2'] = preds_full_df['span2'].apply(format_text)

res_df['full'] = res_df['text'] + ' ' + res_df['span1'] + ' ' + res_df['span2']
preds_full_df['full'] = preds_full_df['text'] + ' ' + preds_full_df['span1'] + ' ' + preds_full_df['span2']

Делаем left join, чтобы соединить голосование и правильные метки для одних и тех же заданий.

In [15]:
new = res_df.merge(preds_full_df.drop(['text', 'span1', 'span2'], axis=1), on='full', how='left')

In [16]:
new_valid = new[new['lb_y'].notna()].copy()
len(new_valid)

259

In [17]:
new_valid.head(5)

,text,span1,span2,lb_x,full,lb_y
0,"Мод и Дора видели, как через прерию несутся по...",Мод и Дора,они появились,False,"Мод и Дора видели, как через прерию несутся по...",False
1,"Мод и Дора видели, как через прерию несутся по...",поезда,они появились,True,"Мод и Дора видели, как через прерию несутся по...",True
2,"Мод и Дора видели, как через прерию несутся по...",клубы,они появились,False,"Мод и Дора видели, как через прерию несутся по...",False
3,"Мод и Дора видели, как через прерию несутся по...",Ревущие звуки,они появились,False,"Мод и Дора видели, как через прерию несутся по...",False
4,"Мод и Дора видели, как через прерию несутся по...",свистки,они появились,False,"Мод и Дора видели, как через прерию несутся по...",False


### Подсчет метрики

Если в правом столбце меток осталось 259 непустых строк, значит, форматирование было подчищено корректно и ничего не потерялось.

In [18]:
new_valid['lb_y'] = new_valid['lb_y'].astype(int)

In [19]:
(new_valid['lb_x'] == new_valid['lb_y']).mean()

0.8378378378378378

`Accuracy = 0.838`